[Based on tweets from Stephen Gruppetta @s_gruppetta_ct](https://twitter.com/s_gruppetta_ct/status/1644735622555504641)

# Object-Oriented Python at the Hogwarts School of Codecraft and Algorithmancy
### Year 7: Final Year (Class Methods | Static Methods )

Students are taller than most professors now–what a change from Year 1! But there's still one Year to go before graduating from Hogwarts

Here are our classes so far

In [1]:
import random 

class Wizard:
    def __init__(self, name, patronus, birth_year):
        self.name = name
        self.patronus = patronus
        self.birth_year = birth_year
        self.wand = None
        self.house = None
        self.skill = 0.2 # from 0.0 (low) to 1.0 (high)

    def assign_wand(self, wand):
        self.wand = wand

    def assign_house(self, house):
        self.house = house
        house.add_member(self)
    
    def increase_skill(self, amount):
        self.skill += amount
        if self.skill > 1.0:
            self.skill = 1.0

    def cast_spell(self, spell):
        if self.wand is not None:
            effect = self.wand.cast_spell(spell, self)
            if effect:
                print(f"{self.name} casts {effect}!")
            else:
                print(f"{self.name} failed to cast {spell.name}!")
        else:
            print(f"{self.name} doesn't have a wand yet!")

    def __gt__(self, other):
        return self.skill > other.skill
    
    def __eq__(self, other):
        have_same_name = self.name == other.name
        have_same_birth_year = self.birth_year == other.birth_year
        have_same_patronus = self.patronus == other.patronus
        return  have_same_name and have_same_birth_year and have_same_patronus

    def __str__(self):
        return self.name
    
    def __repr__(self):
        return f"{type(self).__name__}({self.name}, {self.patronus}, {self.birth_year})"
    
class Student(Wizard):
    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
        self.year = year
        self.subject_grades = {}
    
    def __str__(self):
        return f"{self.name} (House: {self.house} | Year {self.year})"  
    
    def assign_house_using_sorting_hat(self):
        # TODO - Sorting Hat logic
        pass

    def take_exam(self, subject, grade):
        self.subject_grades[subject] = grade

    def assign_subjects(self, subjects):
        self.subject_grades = {subject: None for subject in subjects}   

    
class Professor(Wizard):
    def __init__(self,name,patronus,birth_year,subject=None):
        super().__init__(name,patronus,birth_year)
        self.subject = subject
    
    def assign_wand(self, wand):
        super().assign_wand(wand)
        self.increase_skill(0.2)

    def assess_student(self, student, grade):
        if self.subject in student.subject_grades:
            print( f"{self.name} assessed {student.name} in {self.subject} and gave them a {grade}%!")
            student.take_exam(self.subject, grade)
        else:
            print(f"{self.name} tried to assess {student.name} in {self.subject} but they are not taking that subject!")
            
class House:
    def __init__(self, name, founder, colours, animal):
        self.name = name
        self.founder = founder
        self.colours = colours
        self.animal = animal
        self.members = []
        self.points = 0
    
    def __iter__(self):
        return iter(self.members)

    def __str__(self):
        return self.name

    def add_member(self, member):
        if member not in self.members:
            self.members.append(member)

    def remove_member(self, member):
        self.members.remove(member)

    def update_points(self, points):
        self.points += points
    
    def get_house_details(self):
        return {
            "name": self.name,
            "founder": self.founder,
            "colours": self.colours,
            "animal": self.animal,
            "points": self.points,
        }
    
    
class Wand:
    def __init__(self, wood, core, length, power = 0.5):
        self.wood = wood
        self.core = core
        self.length = length
        self.power = power # from 0.0 (low) to 1.0 (high)

    def cast_spell(self, spell, wizard):
        if spell.is_successful(self, wizard):
            return spell.effect
        return None

class Spell:
    def __init__(self, name, effect, difficulty):
        self.name = name
        self.effect = effect
        self.difficulty = difficulty # from 0.0 (easy) to 1.0 (hard)
    
    def is_successful(self, wand, wizard):
        success_rate = (1 - self.difficulty) * wand.power * wizard.skill
        return random.random() < success_rate

There's one `# TODO` comment in there. It's in `Student.assign_house_using_sorting_hat()`

```python
class Student:
    def __init__(self, name, age, height, weight):
        self.name = name
        self.age = age
        self.height = height
        self.weight = weight
        self.house = None

    ...

    def assign_house_using_sorting_hat(self):
        # TODO - Sorting Hat logic
        pass

    ...
    
```

Let's go through a few versions of this method–not all will be ideal but it will help us see the progression of how we get to a good solution

Here's a first (flawed) attempt

```python
class Student:
    ...

    def assign_house_using_sorting_hat(self):
        houses = [ 
            House("Gryffindor", "Godric Gryffindor", ["scarlet" ,"gold"], "lion"),
            House("Hufflepuff", "Helga Hufflepuff", ["yellow", "black"], "badger"),
            House("Ravenclaw", "Rowena Ravenclaw", ["blue", "bronze"], "eagle"),
            House("Slytherin", "Salazar Slytherin", ["green", "silver"], "serpent"),
            ]
        house = random.choice(houses)
        self.assign_house(house)
    
    ...

```

I'm showing this example first to highlight an issue we need to think about

This method chooses a house at random for a student (Yes, I know, the Sorting Hat wasn't random, but, but…)

Let's try it out and see what happens


In [2]:
import random 

class Wizard:
    def __init__(self, name, patronus, birth_year):
        self.name = name
        self.patronus = patronus
        self.birth_year = birth_year
        self.wand = None
        self.house = None
        self.skill = 0.2 # from 0.0 (low) to 1.0 (high)

    def assign_wand(self, wand):
        self.wand = wand

    def assign_house(self, house):
        self.house = house
        house.add_member(self)
    
    def increase_skill(self, amount):
        self.skill += amount
        if self.skill > 1.0:
            self.skill = 1.0

    def cast_spell(self, spell):
        if self.wand is not None:
            effect = self.wand.cast_spell(spell, self)
            if effect:
                print(f"{self.name} casts {effect}!")
            else:
                print(f"{self.name} failed to cast {spell.name}!")
        else:
            print(f"{self.name} doesn't have a wand yet!")

    def __gt__(self, other):
        return self.skill > other.skill
    
    def __eq__(self, other):
        have_same_name = self.name == other.name
        have_same_birth_year = self.birth_year == other.birth_year
        have_same_patronus = self.patronus == other.patronus
        return  have_same_name and have_same_birth_year and have_same_patronus

    def __str__(self):
        return self.name
    
    def __repr__(self):
        return f"{type(self).__name__}({self.name}, {self.patronus}, {self.birth_year})"
    
class Student(Wizard):
    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
        self.year = year
        self.subject_grades = {}
    
    def __str__(self):
        return f"{self.name} (House: {self.house} | Year {self.year})"  
    
    # New method
    def assign_house_using_sorting_hat(self):
        houses = [ 
            House("Gryffindor", "Godric Gryffindor", ["scarlet" ,"gold"], "lion"),
            House("Hufflepuff", "Helga Hufflepuff", ["yellow", "black"], "badger"),
            House("Ravenclaw", "Rowena Ravenclaw", ["blue", "bronze"], "eagle"),
            House("Slytherin", "Salazar Slytherin", ["green", "silver"], "serpent"),
            ]
        house = random.choice(houses)
        self.assign_house(house)

    def take_exam(self, subject, grade):
        self.subject_grades[subject] = grade

    def assign_subjects(self, subjects):
        self.subject_grades = {subject: None for subject in subjects}   

    
class Professor(Wizard):
    def __init__(self,name,patronus,birth_year,subject=None):
        super().__init__(name,patronus,birth_year)
        self.subject = subject
    
    def assign_wand(self, wand):
        super().assign_wand(wand)
        self.increase_skill(0.2)

    def assess_student(self, student, grade):
        if self.subject in student.subject_grades:
            print( f"{self.name} assessed {student.name} in {self.subject} and gave them a {grade}%!")
            student.take_exam(self.subject, grade)
        else:
            print(f"{self.name} tried to assess {student.name} in {self.subject} but they are not taking that subject!")
            
class House:
    def __init__(self, name, founder, colours, animal):
        self.name = name
        self.founder = founder
        self.colours = colours
        self.animal = animal
        self.members = []
        self.points = 0
    
    def __iter__(self):
        return iter(self.members)

    def __str__(self):
        return self.name

    def add_member(self, member):
        if member not in self.members:
            self.members.append(member)

    def remove_member(self, member):
        self.members.remove(member)

    def update_points(self, points):
        self.points += points
    
    def get_house_details(self):
        return {
            "name": self.name,
            "founder": self.founder,
            "colours": self.colours,
            "animal": self.animal,
            "points": self.points,
        }
    
    
class Wand:
    def __init__(self, wood, core, length, power = 0.5):
        self.wood = wood
        self.core = core
        self.length = length
        self.power = power # from 0.0 (low) to 1.0 (high)

    def cast_spell(self, spell, wizard):
        if spell.is_successful(self, wizard):
            return spell.effect
        return None

class Spell:
    def __init__(self, name, effect, difficulty):
        self.name = name
        self.effect = effect
        self.difficulty = difficulty # from 0.0 (easy) to 1.0 (hard)
    
    def is_successful(self, wand, wizard):
        success_rate = (1 - self.difficulty) * wand.power * wizard.skill
        return random.random() < success_rate

In [3]:
harry = Student("Harry Potter", "stag", 1980)
ron = Student("Ron Weasley", "Jack Russell terrier", 1980)

harry.assign_house_using_sorting_hat()
ron.assign_house_using_sorting_hat()

print(harry.house)
print(ron.house)

Slytherin
Gryffindor


In [4]:
print(harry.house)

Slytherin


In [5]:
print(ron.house.members)

[Student(Ron Weasley, Jack Russell terrier, 1980)]


In [6]:
print( harry.house is ron.house )

False


### Class Methods

Note that houses are chosen at random. I ran this script several times until Harry and Ron where both assigned to the same house to demonstrate this

Note how Harry is the only member of his house even though both he and Ron where assigned to Gryffindor

The objects in `harry.​house` and `ron.​house` are different instances of the class `House`, even though they have the same values for name, founder, and so on

This is because new instances for each house are created each time you call `.assign_house_using_sorting_hat()`



Second attempt: let's pass a sequence with all the houses to the method and we can create the houses elsewhere, not within the method itself
    
```python
class Student(Wizard):
    ...

    def assign_house_using_sorting_hat(self, houses):
        house = random.choice(houses)
        self.assign_house(house)
    
    ...

```

We can create the `House` instances with the other classes, for now

In this run, all the students happened to end up in the same house, Slytherin (it's random allocation!)

Since there's only one Slytherin instance, they're all in the same instance now, not separate ones

In [7]:
class Student(Wizard):
    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
        self.year = year
        self.subject_grades = {}
    
    def __str__(self):
        return f"{self.name} (House: {self.house} | Year {self.year})"  
    
    # New method
    def assign_house_using_sorting_hat(self, houses):
        house = random.choice(houses)
        self.assign_house(house)

    def take_exam(self, subject, grade):
        self.subject_grades[subject] = grade

    def assign_subjects(self, subjects):
        self.subject_grades = {subject: None for subject in subjects}   

In [8]:
houses = [ 
        House("Gryffindor", "Godric Gryffindor", ["scarlet" ,"gold"], "lion"),
        House("Hufflepuff", "Helga Hufflepuff", ["yellow", "black"], "badger"),
        House("Ravenclaw", "Rowena Ravenclaw", ["blue", "bronze"], "eagle"),
        House("Slytherin", "Salazar Slytherin", ["green", "silver"], "serpent"),
        ]

harry = Student("Harry Potter", "stag", 1980)
ron = Student("Ron Weasley", "Jack Russell terrier", 1980)

harry.assign_house_using_sorting_hat(houses)
ron.assign_house_using_sorting_hat(houses)

print(harry.house)
print(ron.house)

Gryffindor
Slytherin


In [9]:
print(harry.house.members)
print(ron.house.members)
print( harry.house is ron.house )

[Student(Harry Potter, stag, 1980)]
[Student(Ron Weasley, Jack Russell terrier, 1980)]
False


This works, although if you're using these houses in separate scripts within a larger project, you have to be careful to define the houses only once and re-use the same instances across your project…

But there's another way...

If we are to define the house instances once, we can do so in a class

Let's create a new module (a new file) called `hogwarts_class.​py` and define a class called `Hogwarts`

This class is a bit different to the ones you've defined so far

Let's look at it in more detail

In [10]:
# hogwarts_class.py

class Hogwarts:
    _houses = {
        "Gryffindor": House("Gryffindor", "Godric Gryffindor", ["scarlet" ,"gold"], "lion"),
        "Hufflepuff": House("Hufflepuff", "Helga Hufflepuff", ["yellow", "black"], "badger"),
        "Ravenclaw": House("Ravenclaw", "Rowena Ravenclaw", ["blue", "bronze"], "eagle"),
        "Slytherin": House("Slytherin", "Salazar Slytherin", ["green", "silver"], "serpent"),
    }

    @classmethod
    def get_house(cls, house_name):
        return cls._houses[house_name]

The class has two attributes: 

* The first is `_houses`. It is a _class attribute_ .
  > Note how it's not linked to `self` as other data attributes we used in previous Years

  It contains a dictionary linking the names of the houses with instances of `House` for each one.
  The leading underscore in `_houses` is not linked to this being a class attribute. It's a convention we use to show this is a **private attribute** and should only be used internally within the class
  
  > It's just a convention and Python doesn't force this behaviour

  This class attribute is not specifically attached to an instance, as the instance variables we've seen before.(It can be accessed directly from the class, such as `Hogwarts._houses`) However, if you create instances of this class, you can also access this data through the instance.

  But in this example we won't create instances of `Hogwarts`. This is also why we don't need an `__init__()` method!

* The second attribute in this class is a method. However, this is not an instance method like the ones we've seen before. The `@​classmethod` before the function definition takes care of this

  You'll notice that the first parameter is not `self` but `cls`

  A class method is not attached to an instance and therefore doesn't need `self`, which refers to the instance

  Instead, a class method is linked to the class and therefore it needs the class as its first parameter. When you call this method, the class is passed implicitly, as you'll see soon

  This is similar to how `self` is the first parameter in an instance method but you don't need to explicitly add the object in the parentheses when you call the method–it's automatically included

  This method accesses `_houses`, which is a dictionary. It uses the dictionary's `.get()` method to get the value associated with the key–this is a `House` object

  We normally prefer to use `get_house()` to access the `_houses` class attribute instead of accessing it directly

You're only creating one instance of each `House`

Therefore, each time you use `Hogwarts.get_house()`, you're fetching one of these four instances


Let's make the necessary changes in `Student` 

The method now accepts a school to indicate the school—this is the name of a class, such as `Hogwarts`

In this example we only have a `Hogwarts` school class, but you could have more schools!

In [11]:
class Student(Wizard):
    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
        self.year = year
        self.subject_grades = {}
    
    def __str__(self):
        return f"{self.name} (House: {self.house} | Year {self.year})"  
    
    # New method
    def assign_house_using_sorting_hat(self, school):
        house_name = random.choice(list(school._houses.keys()))
        house = school.get_house(house_name)
        self.assign_house(house)
        
    def take_exam(self, subject, grade):
        self.subject_grades[subject] = grade

    def assign_subjects(self, subjects):
        self.subject_grades = {subject: None for subject in subjects}   

This method now picks a random house name and fetches the instances created within the `Hogwarts` class. There will only be one instance for each house

But, have a look at that last code snippet again…

We used `school._houses.keys()` in the `Student` class. We're accessing the private attribute `_houses` from the `Hogwarts` class

> Python won't stop you from doing this. But it's best to avoid it. So let's add another class method in `Hogwarts` and then use the class method

In [12]:
# hogwarts_class.py

class Hogwarts:
    _houses = {
        "Gryffindor": House("Gryffindor", "Godric Gryffindor", ["scarlet" ,"gold"], "lion"),
        "Hufflepuff": House("Hufflepuff", "Helga Hufflepuff", ["yellow", "black"], "badger"),
        "Ravenclaw": House("Ravenclaw", "Rowena Ravenclaw", ["blue", "bronze"], "eagle"),
        "Slytherin": House("Slytherin", "Salazar Slytherin", ["green", "silver"], "serpent"),
    }

    @classmethod
    def get_house(cls, house_name):
        return cls._houses[house_name]
    
    @classmethod
    def get_house_names(cls):
        return list(cls._houses.keys())
    

class Student(Wizard):
    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
        self.year = year
        self.subject_grades = {}
    
    def __str__(self):
        return f"{self.name} (House: {self.house} | Year {self.year})"  
    
    # New method
    def assign_house_using_sorting_hat(self, school):
        house_name = random.choice(school.get_house_names())
        house = school.get_house(house_name)
        self.assign_house(house)
        
    def take_exam(self, subject, grade):
        self.subject_grades[subject] = grade

    def assign_subjects(self, subjects):
        self.subject_grades = {subject: None for subject in subjects}   

We can finally import the `Hogwarts` class and use it in the calls to `.assign_house_using_sorting_hat()`

In [13]:
harry = Student("Harry Potter", "stag", 1980)
ron = Student("Ron Weasley", "Jack Russell terrier", 1980)

harry.assign_house_using_sorting_hat(Hogwarts)
ron.assign_house_using_sorting_hat(Hogwarts)

print(harry.house)
print(ron.house)

Gryffindor
Gryffindor


In [14]:
ron.house.members

[Student(Harry Potter, stag, 1980),
 Student(Ron Weasley, Jack Russell terrier, 1980)]

In [15]:
harry.house.members

[Student(Harry Potter, stag, 1980),
 Student(Ron Weasley, Jack Russell terrier, 1980)]

In [16]:
ron.house is harry.house

True

## Unique IDs

Let's have a look at another example with class attributes

You need to assign a unique ID to each student—even wizards need bureaucracy, after all!

You can create a class attribute with the student count
This is an attribute that must belong to the class, not an instance

```python
class Student(Wizard):
    student_count = 0

    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
       
        Student.student_count += 1
        self.id = f"SID{Student.student_count:05d}"
        self.year = year
        self.subject_grades = {}
    
    ...
```

Each time you create a `Student` instance, the `__init__()` method is called. This:

1. Increments the class attribute `Student.​student_count` – note that there's no `self` used on this line. This is a value that belongs to the class as a whole

2. It creates `self.​id` using the class attribute `.student_count`. The instance data attribute `.id` belongs to the instance—each student has a unique ID

You can see this in action here:

In [17]:
class Student(Wizard):
    student_count = 0

    def __init__(self,name,patronus,birth_year,year=1):
        super().__init__(name,patronus,birth_year)
       
        Student.student_count += 1
        self.id = f"SID{Student.student_count:05d}"
        self.year = year
        self.subject_grades = {}
    
    def __str__(self):
        return f"{self.name} (House: {self.house} | Year {self.year})"  
    
    def assign_house_using_sorting_hat(self, school):
        house_name = random.choice(school.get_house_names())
        house = school.get_house(house_name)
        self.assign_house(house)
        
    def take_exam(self, subject, grade):
        self.subject_grades[subject] = grade

    def assign_subjects(self, subjects):
        self.subject_grades = {subject: None for subject in subjects}  

In [18]:
harry = Student("Harry Potter", "stag", 1980)
ron = Student("Ron Weasley", "Jack Russell terrier", 1980)

print(harry.id)
print(ron.id)


SID00001
SID00002


## Static methods

Before wrapping up the Year and your entire studies at Hogwarts School of Codecraft and Algorithmancy, there's one more type of method to discuss

So far, you've seen instance methods and class methods

Before wrapping up the Year and your entire studies at Hogwarts School of Codecraft and Algorithmancy, there's one more type of method to discuss

So far, you've seen **instance methods** and **class methods**

Instance methods belong to the instance of a class. The instance is passed to the method as its first argument, normally called `self`
Therefore, when you call

`harry.​assign_wand(some_wand)`

you're effectively calling

`Student.​assign_wand(harry, some_wand)`

Class methods have access to the class as a whole instead of individual instances. These have the class passed as their first argument instead of the instance

Another type of method is the **`_static method_`** which doesn't have access to the instance or the class


Therefore, static methods do not need a required first parameter such as `self` for instance methods and `cls` for class methods

Here's an example of a static method in the `Wand` class:

```python

class Wand:
    ...

    @staticmethod
    def get_all_wand_makers():
        wand_makers = [ 
            "Garrick Ollivander", 
            "Mykew Gregorovitch",
            "Arturo Cephalopos",
            "Violetta Beauvais",
            "Thiago Quintana",
            ]
        return wand_makers

```

There's `@​staticmethod` before the class definition to indicate what type of method this is

Note that this method doesn't need any information that's stored within the class or an instance of the class

It's a standalone method but it fits well within the `Wand` class

Here's how we can use this method

You can call it directly from the class, as in `Wand.​get_all_wand_makers()`

You could call it from an instance, too, but this has no additional information relating to the instance

In [19]:
class Wand:
    def __init__(self, wood, core, length, power = 0.5):
        self.wood = wood
        self.core = core
        self.length = length
        self.power = power # from 0.0 (low) to 1.0 (high)

    def cast_spell(self, spell, wizard):
        if spell.is_successful(self, wizard):
            return spell.effect
        return None
    
    @staticmethod
    def get_all_wand_makers():
        wand_makers = [ 
            "Garrick Ollivander", 
            "Mykew Gregorovitch",
            "Arturo Cephalopos",
            "Violetta Beauvais",
            "Thiago Quintana",
            ]
        return wand_makers
    


In [20]:
harry = Student("Harry Potter", "stag", 1980)

harry.assign_wand(Wand("holly", "phoenix feather", 11, 0.8))

print(Wand.get_all_wand_makers())

print(harry.wand.get_all_wand_makers())

['Garrick Ollivander', 'Mykew Gregorovitch', 'Arturo Cephalopos', 'Violetta Beauvais', 'Thiago Quintana']
['Garrick Ollivander', 'Mykew Gregorovitch', 'Arturo Cephalopos', 'Violetta Beauvais', 'Thiago Quintana']


This brings us to the end of of Year 7 and of your studies at Hogwarts School of Codecraft and Algorithmancy

You're now a fully qualified wizard or witch…

…and more importantly, hopefully you have a better understanding of classes and OOP than before you started!
